In [1]:
# # TensorFlow and tf.keras
import tensorflow as tf
import pandas as pd

#if working with tensorflow > v2 uncomment all of the following
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import utils


#if working with tf < v2 comment above and uncomment next block of code

# import keras
# import keras.backend
# from keras import layers
# from keras import models
# import keras.utils

print(keras.__version__)
print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import time
import h5py
import sklearn
from sklearn.utils import shuffle

import os

2.2.4-tf
2.0.0


Import data from RNN_data.npz

In [2]:
# here we input the dummy data I haven't made yet

In [3]:
# training data
traindata_sig = np.load('data/dummyvectors_signal.npz')
traindata_bkg = np.load('data/dummyvectors_bkg.npz')
trainlabels = np.concatenate((traindata_sig['labels'],traindata_bkg['labels']),axis=0)
trainvectors = np.concatenate((traindata_sig['fourvectors'], traindata_bkg['fourvectors']), axis = 1)
trainradii = np.concatenate((traindata_sig['radiilab'], traindata_bkg['radiilab']), axis = 0)
traintheta = np.concatenate((traindata_sig['thetalab'], traindata_bkg['thetalab']), axis = 0)
trainz = np.concatenate((traindata_sig['zlab'], traindata_bkg['zlab']), axis = 0)
n_train = len(trainlabels)
trainlabels = trainlabels.reshape(int(n_train/100), 100, 2)
trainvectors = trainvectors.reshape(int(n_train/100), 100, 4)

# testing data
testdata_sig = np.load('data/dummyvectors_sgn_test.npz')
testdata_bkg = np.load('data/dummyvectors_bkg_test.npz')
testlabels = np.concatenate((testdata_sig['labels'], testdata_bkg['labels']),axis=0)
testvectors = np.concatenate((testdata_sig['fourvectors'], testdata_bkg['fourvectors']), axis = 1)
testradii = np.concatenate((testdata_sig['radiilab'], testdata_bkg['radiilab']), axis = 0)
test_theta = np.concatenate((testdata_sig['thetalab'], testdata_bkg['thetalab']), axis = 0)
testz = np.concatenate((testdata_sig['zlab'], testdata_bkg['zlab']), axis = 0)
n_test = len(testlabels)
testlabels = testlabels.reshape(int(n_train/20), 20, 2)
testvectors = testvectors.reshape(int(n_train/20), 20, 4)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [4]:
def build_model(rnn_units, batch, n_train):
    input1 = layers.Input(name = 'vector_input', batch_shape=(None, batch, 4))
    x = layers.Dense(64, activation=tf.nn.relu, name = 'relu1')(input1)
    x = layers.GRU(4*rnn_units, return_sequences = True, name = 'gru1')(x)
    x = layers.GRU(4*rnn_units, return_sequences = True, name = 'gru2')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GRU(rnn_units, return_sequences = True, name = 'gru3')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GRU(rnn_units, return_sequences = True, name = 'gru4')(x)
#     xf = layers.Flatten(name = "flatten1")(x1)
#     input2 = layers.Input(shape=(1,), name = 'z_input')
#     input3 = layers.Input(shape=(2,), name = 'radii_input')
#     input4 = layers.Input(shape=(1,), name = 'theta_input')
#     x = layers.concatenate(inputs = [xf, input2, input3, input4],axis=-1, name = 'concat')
    output = layers.Dense(2, activation=tf.nn.softmax, name = 'sigmoid')(x)
    model = models.Model(inputs=input1, outputs=output)
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['CategoricalAccuracy', 'accuracy'])
    return model

In [5]:
rnn_units = 512
batch = 20
RNN = build_model(rnn_units, batch, 10)

In [6]:
example_batch = trainvectors[:10,:,:]
example_labels = trainlabels[:10,:,:]
example_radii = trainradii[:10]
example_theta = traintheta[:10]
example_z = trainz[:10]
print(example_radii.shape)
example_result = RNN.predict(x = [example_batch])
results = RNN.evaluate(x = [example_batch], y = example_labels)
print(example_result)
print(results)

(10, 2)
10/1 [============================================================================================================================================================================================================================================================================================================] - 2s 238ms/sample - loss: 0.4853 - CategoricalAccuracy: 0.9800 - accuracy: 0.9800
[[[0.5152952  0.48470485]
  [0.55339664 0.4466033 ]
  [0.6101467  0.3898533 ]
  [0.65099525 0.34900475]
  [0.66915023 0.3308497 ]
  [0.6697819  0.33021805]
  [0.66451806 0.33548197]
  [0.6430293  0.35697076]
  [0.5995239  0.40047613]
  [0.5742403  0.42575967]
  [0.5763983  0.42360175]
  [0.5923068  0.40769318]
  [0.5985005  0.40149954]
  [0.6104254  0.3895745 ]
  [0.61777276 0.3822273 ]
  [0.6320924  0.36790755]
  [0.643527   0.356473  ]
  [0.63735664 0.36264336]
  [0.6245165  0.37548354]
  [0.60967755 0.39032245]]

 [[0.50383586 0.49616408]
  [0.5209544  0.47904563]
  [0.5574086  0.44259146]
  

In [7]:
RNN = build_model(rnn_units, batch, n_train)

In [ ]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print('')
        print('.', end='')
    
checkpoint_path = "model/RNN_2x2048_2x512_1batchrenorm.h5"
if not os.path.exists("model"):
    os.mkdir("model")

# Create checkpoint callback
# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
#                                                  save_best_only=True,
#                                                  verbose=1)
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', 
                                   verbose=1, save_best_only=True, 
                                   save_weights_only=False, mode='auto')    
EPOCHS = 1000
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = RNN.fit(
  trainvectors, trainlabels,
  epochs=EPOCHS, validation_split = 0.2, verbose = 0,
  callbacks=[early_stop, model_checkpoint])


Epoch 00001: val_loss improved from inf to 1.00977, saving model to model/RNN_2x2048_2x512_1batchrenorm.h5

Epoch 00002: val_loss did not improve from 1.00977


In [ ]:
!ls model
best_model = keras.models.load_model('model/RNN_2x2048_2x512_1batchrenorm.h5')
best_model.summary()
results = best_model.evaluate(testvectors, testlabels, verbose = 0)
print(results)

In [ ]:
# Plot ROC
predict = best_model.predict(trainvectors)

print(predict[0])
print(predict[:,:,0].flatten())
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(trainlabels[:,:,0].flatten(), predict[:,:,0].flatten())
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, lw=2, color='blue', label='auc = %.3f' % (roc_auc))
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='k', label='random chance')
plt.xlim([0, 1.0])
plt.ylim([0, 1.0])
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.title('receiver operating curve')
plt.legend(loc="lower right")
plt.show()